In [1]:
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
from sklearn.preprocessing import StandardScaler
import geopandas as gpd
import os
import warnings
from datetime import datetime
import numpy as np
import panel as pn
import geopandas as gpd

warnings.simplefilter("ignore")
rcParams['axes.unicode_minus'] = False

font_path = r'C:/Users/user/NanumFontSetup_TTF_ALL/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=font_path, size=18).get_name()
rc('font',family=font_name)

## 1. Data Loading

In [15]:
file_path = '../data/건물 리스트/201912_건물DB_전체분/build_incheon.txt'

f = open(file_path)
df = pd.DataFrame()
i = 0

with open(file_path) as f:
    for line in f.readlines():
        # 라인하나 불러옴
        line = line.replace("|",",")
    
        # 콤마를 기준으로 데이터를 나눔
        val_split = line.split(sep=',')
    
        if val_split[2] != '부평구':
            continue
    
        df_tmp = pd.DataFrame.from_dict([{'시도명':val_split[1],
                              '시군구명':val_split[2],
                              '법정읍면동명':val_split[3],
                              '법정리명':val_split[4],
                              '본번':val_split[6],
                              '부번':val_split[7],
                              '기초구역번호':val_split[27]}])
        
    
        df = pd.concat([df,df_tmp],axis=0,ignore_index=True)
        i += 1
        
        if i == 1000:
            display(val_split[2])
            display(i)

df.head()

'부평구'

1000

,시도명,시군구명,법정읍면동명,법정리명,본번,부번,기초구역번호
0,인천광역시,부평구,부평동,,276,74,21405
1,인천광역시,부평구,부평동,,274,85,21405
2,인천광역시,부평구,부평동,,274,94,21405
3,인천광역시,부평구,부평동,,274,93,21405
4,인천광역시,부평구,부평동,,274,56,21405


In [16]:
df.shape

(34861, 7)

In [22]:
df = df.drop_duplicates(list(df),keep='first').reset_index(drop=True)
df.drop('index',axis=1,inplace=True)

In [23]:
df

,시도명,시군구명,법정읍면동명,법정리명,본번,부번,기초구역번호
0,인천광역시,부평구,부평동,,276,74,21405
1,인천광역시,부평구,부평동,,274,85,21405
2,인천광역시,부평구,부평동,,274,94,21405
3,인천광역시,부평구,부평동,,274,93,21405
4,인천광역시,부평구,부평동,,274,56,21405
...,...,...,...,...,...,...,...
23988,인천광역시,부평구,구산동,,63,2,21457
23989,인천광역시,부평구,구산동,,39,1,21417
23990,인천광역시,부평구,구산동,,34,1,21417
23991,인천광역시,부평구,구산동,,9,5,21417


In [7]:
store_path = os.path.join(cwd,'data/전처리 파일/부평구 건물 리스트.csv')
df.to_csv(store_path)

## 2. 개별공시지가 전처리

In [8]:
# before_path = '../data/전처리 파일/개별공시지가(수정본).csv'
before_path = '../data/전처리 파일/건물 개별공시가격.csv'
before_data = pd.read_csv(before_path)
# after_data = pd.read_csv(after_path)

In [9]:
before_data.drop('법정리명',axis=1,inplace=True)

for i in range(6,9):
    before_data.iloc[23935,i] = 0

before_data.iloc[23935]

시도명       인천광역시
시군구명        부평구
법정읍면동명      구산동
본번           66
부번            6
기초구역번호    21417
2017          0
2018          0
2019          0
Name: 23935, dtype: object

In [10]:
def correct_bas_id(data):
    change_row_idx = data[data['기초구역번호'].isin({'0','1','오피스텔','한국아파트'})].index[1:]
    
    # 옳지 않은 기초구역번호 수정
    basis_area_dict = {'3258':'21404','17700':'21313','18483':'21318','18968':'21320','18970':'21320'
                  ,'19904':'21333'}
    
    for idx in change_row_idx:
        data.iloc[idx,5] = basis_area_dict[str(idx)]
    
    data = data.drop(1542,axis=0)
    data = data.reset_index(drop=True)
    return data

# 3258, 21404
# 17700, 21313
# 18483,21318
# 18968, 21320
# 18970, 21320
# 19904, 21333

In [12]:
def remove_char(price):
    table = str.maketrans(',원','  ')
    result = price.translate(table).replace(' ','')
    return result

def correct_price(price_data):
    for col in range(6,9):
        price_data.iloc[:,col] = price_data.iloc[:,col].astype(str)
        price_data.iloc[:,col] = price_data.iloc[:,col].apply(lambda x : remove_char(x)).astype(int)
    
    return price_data

In [13]:
before_data = correct_price(correct_bas_id(before_data))
# after_data = correct_price(correct_bas_id(after_data))
display(before_data)
# display(after_data)

,시도명,시군구명,법정읍면동명,본번,부번,기초구역번호,2017,2018,2019
0,인천광역시,부평구,부평동,276,74,21405,4211000,4060000,4060000
1,인천광역시,부평구,부평동,274,85,21405,4211000,4211000,4211000
2,인천광역시,부평구,부평동,274,94,21405,4083000,4083000,4083000
3,인천광역시,부평구,부평동,274,93,21405,4150000,4264000,4392000
4,인천광역시,부평구,부평동,274,56,21405,4150000,4264000,4392000
...,...,...,...,...,...,...,...,...,...
23987,인천광역시,부평구,구산동,63,2,21457,655500,721200,958800
23988,인천광역시,부평구,구산동,39,1,21417,799700,846000,904000
23989,인천광역시,부평구,구산동,34,1,21417,839700,897100,1226000
23990,인천광역시,부평구,구산동,9,5,21417,815500,862800,912600


In [14]:
before_data['기초구역번호'].unique()

array(['21405', '21387', '21404', '21389', '21395', '21394', '21393',
       '21388', '21390', '21379', '21359', '21358', '21366', '21354',
       '21403', '21400', '21432', '21410', '21428', '21409', '21408',
       '21424', '21407', '21406', '21425', '21414', '21426', '21378',
       '21360', '21391', '21353', '21397', '21456', '21411', '21356',
       '21355', '21352', '21351', '21396', '21429', '21430', '21431',
       '21412', '21392', '21399', '21365', '21357', '21364', '21458',
       '21449', '21448', '21447', '21442', '21441', '21375', '21440',
       '21439', '21436', '21435', '21413', '21455', '21446', '21453',
       '21454', '21450', '21445', '21437', '21438', '21452', '21451',
       '21434', '21444', '21433', '21443', '21384', '21367', '21369',
       '21308', '21309', '21313', '21361', '21380', '21376', '21381',
       '21377', '21386', '21362', '21363', '21307', '21368', '21306',
       '21305', '21382', '21385', '21383', '21374', '21373', '21372',
       '21370', '213

In [93]:
gap = before_data.iloc[:,6:9] - after_data.iloc[:,6:9]
gap = gap.astype(int)
null_index = gap[gap['2017'] < 0].index

In [94]:
display(before_data.iloc[null_index].head())
display(after_data.iloc[null_index].head())

,시도명,시군구명,법정읍면동명,본번,부번,기초구역번호,2017,2018,2019,차분평균,차분평균(정규화)
12260,인천광역시,부평구,십정동,53,0,21447,0,1019000,989400,0.0,-0.720723
12261,인천광역시,부평구,십정동,56,1,21447,863000,890000,889400,0.0,-0.720723
12262,인천광역시,부평구,십정동,56,2,21447,880300,907800,907800,0.0,-0.720723
12264,인천광역시,부평구,십정동,27,0,21447,1154000,1039000,1057000,0.0,-0.720723
12266,인천광역시,부평구,십정동,54,3,21447,880300,907800,880200,0.0,-0.720723


,시도명,시군구명,법정읍면동명,본번,부번,기초구역번호,2017,2018,2019
12260,인천광역시,부평구,십정동,53,0,21447,3943000,3943000,3943000
12261,인천광역시,부평구,십정동,56,1,21447,3943000,3943000,3943000
12262,인천광역시,부평구,십정동,56,2,21447,4057000,4057000,4057000
12264,인천광역시,부평구,십정동,27,0,21447,4240000,4240000,4240000
12266,인천광역시,부평구,십정동,54,3,21447,4240000,4240000,4240000


- before data도 after data와 같이 처음에는 같은 오류가 발생했었다. 
- 그래서 before data는 차분 평균이 0인 데이터들을 다시 크롤링을 진행했었고 데이터가 3개년 데이터가 같은 것들은 없어졌었다.
- 따라서 after data도 결국에는 같은 결과를 보일 것이며 이전의 before data를 그대로 사용해도 된다는 결론을 낼수 있다.
- 이외의 것들은 모두 같기 때문이다.

### 2.1 cleaning 결과 확인

In [9]:
basis_area = np.sort(df['기초구역번호'].unique())
basis_area

array(['21300', '21301', '21302', '21303', '21304', '21305', '21306',
       '21307', '21308', '21309', '21310', '21311', '21312', '21313',
       '21314', '21315', '21316', '21317', '21318', '21319', '21320',
       '21321', '21322', '21323', '21324', '21325', '21326', '21327',
       '21328', '21329', '21330', '21331', '21332', '21333', '21334',
       '21335', '21336', '21337', '21338', '21339', '21340', '21341',
       '21342', '21343', '21344', '21345', '21346', '21347', '21348',
       '21349', '21350', '21351', '21352', '21353', '21354', '21355',
       '21356', '21357', '21358', '21359', '21360', '21361', '21362',
       '21363', '21364', '21365', '21366', '21367', '21368', '21369',
       '21370', '21371', '21372', '21373', '21374', '21375', '21376',
       '21377', '21378', '21379', '21380', '21381', '21382', '21383',
       '21384', '21385', '21386', '21387', '21388', '21389', '21390',
       '21391', '21392', '21393', '21394', '21395', '21396', '21397',
       '21398', '213

### 2.2 차분평균 생성

In [299]:
pd.set_option("max_r", 20)

In [95]:
before_data['차분평균']= [round(np.diff(list(row[6:9])).mean(),3) for idx, row in before_data.iterrows()]
before_data['차분평균']

0        -75500.0
1             0.0
2             0.0
3        121000.0
4        121000.0
           ...   
23987    151650.0
23988     52150.0
23989    193150.0
23990     48550.0
23991     50000.0
Name: 차분평균, Length: 23992, dtype: float64

In [2]:
pd.set_option("max_rows",100)

In [96]:
before_data[before_data['차분평균'] == 0]

,시도명,시군구명,법정읍면동명,본번,부번,기초구역번호,2017,2018,2019,차분평균,차분평균(정규화)
1,인천광역시,부평구,부평동,274,85,21405,4211000,4211000,4211000,0.0,-0.720723
2,인천광역시,부평구,부평동,274,94,21405,4083000,4083000,4083000,0.0,-0.720723
110,인천광역시,부평구,부평동,160,71,21395,0,0,0,0.0,-0.720723
112,인천광역시,부평구,부평동,160,72,21395,0,0,0,0.0,-0.720723
114,인천광역시,부평구,부평동,160,2,21395,0,0,0,0.0,-0.720723
1196,인천광역시,부평구,부평동,733,2,21407,0,0,0,0.0,-0.720723
12278,인천광역시,부평구,산곡동,310,95,21405,3943000,3943000,3943000,0.0,-0.720723
17781,인천광역시,부평구,청천동,12,10,21306,0,0,0,0.0,-0.720723
18407,인천광역시,부평구,삼산동,426,10,21323,2980000,2980000,2980000,0.0,-0.720723
23931,인천광역시,부평구,구산동,8,2,21417,0,0,0,0.0,-0.720723


In [97]:
price_data.columns

Index(['시도명', '시군구명', '법정읍면동명', '본번', '부번', '기초구역번호', '2017', '2018', '2019',
       '차분평균', '차분평균(정규화)'],
      dtype='object')

In [98]:
all_zero = []

for idx, row in price_data.iterrows():
    year_17 = row[6]
    year_18 = row[7]
    year_19 = row[8]
    
    if (year_17+year_18+year_19) == 0:
        all_zero.append(idx)

display(price_data.iloc[all_zero])
display(len(all_zero))

,시도명,시군구명,법정읍면동명,본번,부번,기초구역번호,2017,2018,2019,차분평균,차분평균(정규화)
110,인천광역시,부평구,부평동,160,71,21395,0,0,0,0.0,-0.720723
112,인천광역시,부평구,부평동,160,72,21395,0,0,0,0.0,-0.720723
114,인천광역시,부평구,부평동,160,2,21395,0,0,0,0.0,-0.720723
1196,인천광역시,부평구,부평동,733,2,21407,0,0,0,0.0,-0.720723
17781,인천광역시,부평구,청천동,12,10,21306,0,0,0,0.0,-0.720723
23931,인천광역시,부평구,구산동,8,2,21417,0,0,0,0.0,-0.720723
23934,인천광역시,부평구,구산동,66,6,21417,0,0,0,0.0,-0.720723


7

In [99]:
price_remove_zero = before_data.drop(all_zero,axis=0).reset_index(drop=True)
price_remove_zero.isnull().sum()

시도명          0
시군구명         0
법정읍면동명       0
본번           0
부번           0
기초구역번호       0
2017         0
2018         0
2019         0
차분평균         0
차분평균(정규화)    0
dtype: int64

In [100]:
price_remove_zero['기초구역번호'].unique()

array(['21405', '21387', '21404', '21389', '21395', '21394', '21393',
       '21388', '21390', '21379', '21359', '21358', '21366', '21354',
       '21403', '21400', '21432', '21410', '21428', '21409', '21408',
       '21424', '21407', '21406', '21425', '21414', '21426', '21378',
       '21360', '21391', '21353', '21397', '21456', '21411', '21356',
       '21355', '21352', '21351', '21396', '21429', '21430', '21431',
       '21412', '21392', '21399', '21365', '21357', '21364', '21458',
       '21449', '21448', '21447', '21442', '21441', '21375', '21440',
       '21439', '21436', '21435', '21413', '21455', '21446', '21453',
       '21454', '21450', '21445', '21437', '21438', '21452', '21451',
       '21434', '21444', '21433', '21443', '21384', '21367', '21369',
       '21308', '21309', '21313', '21361', '21380', '21376', '21381',
       '21377', '21386', '21362', '21363', '21307', '21368', '21306',
       '21305', '21382', '21385', '21383', '21374', '21373', '21372',
       '21370', '213

In [101]:
result = price_remove_zero.groupby('기초구역번호')['차분평균'].mean().to_frame().reset_index()
result.head()

,기초구역번호,차분평균
0,21300,31337.735849
1,21301,344640.000000
2,21302,174949.462366
3,21303,97041.791045
4,21304,166589.336493


## 3. 기초구역정보 로드

In [103]:
basis_area = gpd.read_file('../data/전처리 파일/부평구_기초구역_5181/부평구_기초구역_5181.shp')
basis_area

,BAS_AR,BAS_ID,BAS_MGT_SN,CTP_KOR_NM,MVMN_DE,MVMN_RESN,NTFC_DE,OPERT_DE,SIG_CD,SIG_KOR_NM,geometry
0,2.648838,21459,2823700025,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((176641.304 442645.152, 176647.527 44..."
1,0.156212,21384,2823700061,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((174332.882 443549.806, 174341.717 44..."
2,0.076024,21403,2823700058,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((176460.207 443392.656, 176503.040 44..."
3,0.527205,21447,2823700035,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((173371.315 442923.042, 173431.784 44..."
4,0.073898,21421,2823700036,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((176813.663 442559.317, 176802.418 44..."
...,...,...,...,...,...,...,...,...,...,...,...
156,0.055146,21423,2823700022,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((176671.251 442643.859, 176686.836 44..."
157,0.117664,21441,2823700023,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((173892.886 442465.677, 173876.623 44..."
158,0.069431,21446,2823700003,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((173718.851 441369.606, 173717.873 44..."
159,0.093812,21453,2823700004,인천광역시,None,None,20121221,None,28237,부평구,"POLYGON ((173523.162 441167.346, 173522.717 44..."


In [104]:
geo_data = basis_area[['BAS_ID','geometry']]
geo_data

,BAS_ID,geometry
0,21459,"POLYGON ((176641.304 442645.152, 176647.527 44..."
1,21384,"POLYGON ((174332.882 443549.806, 174341.717 44..."
2,21403,"POLYGON ((176460.207 443392.656, 176503.040 44..."
3,21447,"POLYGON ((173371.315 442923.042, 173431.784 44..."
4,21421,"POLYGON ((176813.663 442559.317, 176802.418 44..."
...,...,...
156,21423,"POLYGON ((176671.251 442643.859, 176686.836 44..."
157,21441,"POLYGON ((173892.886 442465.677, 173876.623 44..."
158,21446,"POLYGON ((173718.851 441369.606, 173717.873 44..."
159,21453,"POLYGON ((173523.162 441167.346, 173522.717 44..."


In [108]:
merge_data = geo_data.merge(result,how='inner',right_on='기초구역번호',left_on='BAS_ID')
merge_data = merge_data.sort_values(by='BAS_ID').reset_index(drop=True)
merge_data.head()

,BAS_ID,geometry,기초구역번호,차분평균
0,21300,"POLYGON ((172624.863 447152.276, 172670.940 44...",21300,31337.735849
1,21301,"POLYGON ((173295.105 447039.530, 173295.147 44...",21301,344640.000000
2,21302,"POLYGON ((173891.544 446881.024, 173891.543 44...",21302,174949.462366
3,21303,"POLYGON ((173668.839 446507.436, 173658.833 44...",21303,97041.791045
4,21304,"POLYGON ((173891.543 446881.019, 173891.500 44...",21304,166589.336493


In [109]:
merge_data.to_csv('../data/전처리 파일/기초구역별_개별공시지가.csv',index=False)